In [1]:
![image.png](attachment:image.png)

'[image.png]' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
![image.png](attachment:image.png) ![image-2.png](attachment:image-2.png)

'[image.png]' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# definitions

beam = {
    "Area"     : 9880  , #mm2
    "d"        : 450   , #mm
    "h"        : 378.8 , #mm
    "b_t"      : 190   , #mm
    "t_flange" : 14.6  , #mm
    "t_web"    : 9.4   , #mm
    "I_x"      : 33740 , #cm4
}

sac  = {
    "h_r"   : 50, #mm görseldeki h, hadve yüksekliği
    "w_r"   : 165, #mm
    "t_sac" : 1.2 #mm
}

anchorage = {
    "r_f" : 19, #mm
    "h_a" : 100, #mm
    "Fu"  : 450 #N/mm2
}

f_ck = 30 #N/mm2
Fy = 355 #N/mm2
Fu = 510 #N/mm2
Pg = 28 #kN
Pq = 10.5 #kN
L = 9 #m
wg = 1. #kN/m
h_c_üs = 80 #mm net beton örtüsü
h_c = h_c_üs + sac["h_r"]

In [5]:
def calc_stud_full_interaction(b_f : float, d : float, beam_length : float,beam_As : float,beam_aralik : float,f_ck : float,Y_con:float,Fy : float =345):
    """
    Tam etkileşimli kompozit kiriş stud çivisi sayısını hesaplar

    Arguments:
        b_f -- Çelik kiriş başlık genişliği mm
        d -- Çelik kiriş yüksekliği mm
        beam_length -- Kiriş uzunluğu m
        beam_As -- Çelik kiriş alanı mm2
        beam_aralik -- Çelik kirişlerin aralıkları m
        f_ck -- Beton karakteristik dayanımı Mpa 
        Y_con -- Beton örtüsü yüksekliği mm

    Keyword Arguments:
        Fy -- Çelik beklenen akma dayanımı (default: {345})

    Returns:
        Tasarım Momenti
    
    Extras:
        b1 ve b2 için eşit aralık kabulü yapıldı
    """
    T = beam_As*Fy /10**3  #kN
    b1 = 2 * beam_length * 1/8 
    b2 = 2 * beam_aralik * 1/2

    b = min(b1,b2) 

    alpha = T / (0.85 * f_ck * b) 
    A_conc = Y_con * b 
    Y2 = Y_con - 0.5*alpha 
    Y1 = (beam_As * Fy - (0.85 * f_ck * A_conc)) / (2 * Fy * b_f) 
    
    
    x = (0.85 * f_ck * A_conc *(Y1 + Y2)) / 10**6 
    y = (2* Fy * b_f *Y1 * (Y1 /2)) / 10**6 
    z = (beam_As * Fy * (d /2 - Y1)) / 10**6 

    Phi_Mn = ( x + y + z) * 0.9  

    return Phi_Mn

def calc_deflection(Beam : list,L : float , Pg : float,Pq : float,w_g : float = 1,E : float = 20_000):
    delta_1 = ((2 * Pg * L**3 * 10**3 / (28*E*Beam[6]*10**4)) + (5*w_g*L**4/(384*E*Beam[6]*10**4)))/10
    delta_2 = (2* Pq * L**3 * 10**3 / (28*E*Beam[6]*10**4))/10
    print(f"{delta_1} + {delta_2} = {delta_1+delta_2}")
    delta_yapım = delta_1 + delta_2

    return delta_yapım

def check_deflection(Beam : list,L : float , Pg : float,Pq : float,w_g : float = 1.,E : float = 20_000) :
    delta_1 = ((2 * Pg * L**3 * 10**3 / (28*E*Beam[6]*10**4)) + (5*w_g*L**4/(384*E*Beam[6]*10**4)))/10
    delta_2 = (2* Pq * L**3 * 10**3 / (28*E*Beam[6]*10**4))/10

    delta_yapım = delta_1 + delta_2

    if delta_1 > L/360:
        print(f"Sehim sınırı sağlanamadı => {delta_1}>{L/360}\nBu koşulun sağlanamadığı durumda çelik kirişe ters sehim verilmesi veya yapım aşamasında geçici destek kullanımı ile çelik kirişin düşey yerdeğiştirmesi azaltılabilir. (Bkz. Kaynak Yayınlar, AISC Design Guide 3).")
    if delta_yapım > L/300:
        print(f"Toplam sehim sınırı aşıldı kesit büyütülmeli veya mesnet koşulları değiştirilmeli.{delta_yapım} > {L/300} ")
    else:
        print(f"Sehim kontrolü sağlandı {delta_yapım} <= {L/300}. \nAyrıca, beton gerekli dayanımına ulaşıncaya kadar, \nçelik enkesitin yapım aşamasındaki sabit ve hareketli yükler altındaki eğilme dayanımı, \nYönetmelik Bölüm 9 da verilen kurallar gözönüne alınarak kontrol edilmelidir")

def yapim_asamasi_sehimi(Pg : float, Pq : float, L : float, Ix : float, E:float = 200_000, wg: float = 1.) -> float:
    """Basit mesnetli kompozit kiriş için yapım aşaması sehim hesabı

    Args:
        Pg (float): Ölü yük
        L (float): Kiriş uzunluğu
        Ix (float): Kiriş atalet momenti
        E (float, optional): Kiriş malzemesinin elastisite modülü. Defaults to 200_000. birim : N/mm2
        wg (float, optional): Kiriş ağırlığı birim:kN/m. Defaults to 1. 

    Returns:
        delta_1 (float): Yapım aşamasındaki sehim
    """

    #Ölü yük etkisi
    a = 2*(Pg * L**3 / (28 * E * Ix))
    b = (5 * wg * L**4 / (384 * E * Ix))
    delta_1 = a + b

    if delta_1 > (L / 360):
        print(f"Ölü yük durumunda sehim sınırı sağlanamadı => {round(delta_1,2)}mm > {round(L/360,2)}mm çelik kirişe ters sehim verilmesi veya yapım aşamasında geçici destek kullanımı ile çelik kirişin düşey yerdeğiştirmesi azaltılabilir.")

    #hareketli yük etkisi
    delta_2 = (2 * Pq * L **3 / (28 * E * Ix))

    #total etki
    delta = delta_1 + delta_2

    if delta > (L/300):
        print(f"Yapım aşaması sehim sınırı sağlanamadı kiriş boyutu arttırılabilir. => {round(delta,2)}mm > {round(L/300,2)}mm" )
    return round(delta,2)

def beton_alanı(h_c_üs : float, b_eff : float, h_r : float) -> float:
    """Yaklaşık beton örtü alanı

    Returns:
        Ac float: net beton örtü alanı
    """
    Ac = (b_eff * h_c_üs) + (0.5 * b_eff * h_r)
    print(f"Ac = ({b_eff} * {h_c_üs}) + (0.5 * {b_eff} * {h_r}) = {Ac}mm2")
    return Ac

def kompozit_kiris_eğilme_kapasitesi(Fy : float, Beam : dict, f_ck : float, Ac : float, b_eff : float) -> float:
   
    C_conc  = 0.85 * f_ck * Ac
    C_steel = Fy * Beam["Area"]
    

    if C_conc < C_steel : 
        print(f"Kompozit döşeme kısmi etkileşimlidir. Plastik tarafsız eksen Çelik kirişin başlık veya gövdesindedir.")
        M_n = 0.

    else:
        print("Plastik tarafsız eksen beton bloğun içindedir. Tam etkileşimli")

        C = min(C_conc,C_steel) #N
        print(f"C ={C} ")

        a = C/(0.85 * f_ck * b_eff)
        print(f"a = {a}")
        
        y = Beam["d"]/2 + (h_c - a/2) 
        print(f"y = {y}")

        M_n = C * y

    return round(M_n,2)

def Kompozit_kirişe_gelen_egilme_momenti(Pu : float):
    pass


In [21]:
def StudCheck(Ds : float, tf : float, Hs : float)-> bool:

    if Ds > 2.5*tf or Ds > 19:
        return False
    if Hs < 4*Ds:
        return False
    return True

def StudSpaceCheck(Ds : float, s : float, Ycon : float) -> bool:

    if s < 6*Ds or s > 8*Ycon or s >914:
        return False
    return True

def DistanceBetweenTwoStudsCheck(x : float, Ds : float)-> bool:

    if x < 4*Ds:
        return False
    return True

def HadveUzerindeKalanStudBoyutlarıKontrolu(hr : float, hs: float, hconc : float) -> bool:

    if hs-hr < 38 or hconc - hs < 13:
        return False
    return True

def MetalDeckCheck(hr : float, wr : float)-> bool:
    if hr > 76 or wr < 50 :
        return False
    return True

def ConcAvailableStressCheck(f_ck : float) -> bool:
    if 20 <= f_ck and f_ck<70:
        return True
    else:
        return False

def SteelAvailableStressCheck(Fy : float) -> bool:
    if Fy <= 460:
        return True
    else:
        print(f"Çelik sınıfı yönetmelik şartını sağlamamaktadır => {Fy} > {460} ")
        False

def ConcreteYoungModules(fck : float,wc : float = 2320)-> float:
    """_summary_

    Args:
        fck (float): MPa
        wc (float, optional): . Defaults to 2320 kg/m^3.

    Returns:
        float: _description_
    """
    Ec = 0.043 * wc **1.5 * fck**0.5
    return Ec

def RatioYoungModules(Ec : float,Es : float = 2*10**5)-> float:
    """_summary_

    Args:
        Ec (float): MPa
        Es (float, optional): MPa. Defaults to 2*10**5.
        

    Returns:
        float: _description_
    """
    return round(Es/Ec,3)

def ChengedConcToSteelArea(n : float, Ac)-> float:
    return round(Ac/n,3)

def CrushConcCapacity(fck : float, Ac : float)-> float:
    return round(0.85*fck*Ac,3)

def YieldBeamCapacity(fy : float, As : float)-> float:
    return round(fy*As,3)

def OneStudShearCapacity(Asa : float, fck : float, Ec : float, Rg : float, Rp : float, Fu : float = 448)-> float:
    Qn = 0.5 * Asa * (fck*Ec)**0.5
    TrashHold = Rg*Rp*Asa*Fu
    if Qn > TrashHold:
        Qn = TrashHold
    return round(TrashHold,3)

def ShearStudCapacity(N : int, Qn : float)-> float:
    return round(N*Qn,3)

def efektif_doseme_genisligi(L : float, Lu : float) -> float:
    """Efektif kompozit döşeme genişliği hesabı

    Args:
        L (float): Kiriş uzunluğu birim : m
        Lu (float): Kiriş aralığı birim : m

    Returns:
        b_eff float: efektif döşeme genişliği birim : m
    """
    b1 = L/8
    b2 = Lu/2

    b = 2*min(b1,b2)

    return b

def Calc_hc(hr : float, Ycon : float,IsParallel : bool)-> float:
    if IsParallel:
        hc = Ycon - (hr/2)
    else:
        hc = Ycon - hr

    return hc

def Calc_I_tr(n : float,b_eff : float, hr : float, hc : float, hb : float,Ab : float, Ibeam : float):

    Ac = b_eff * hc
    print(f"Ac ={Ac}")

    Act = Ac/n
    print(f"Act ={Act}")

    Ict = b_eff * hc**3 / (12 * n)
    print(f"Ict = {Ict}")

    y_beam = hc + hr + hb/2
    y_conc = hc / 2
    print(f"y_beam = {y_beam}, y_conc ={y_conc}")

    TotalAy = Act*y_conc + Ab * y_beam
    TotalA = Act + Ab
    print(f"TotalAy = {TotalAy}, TotalA = {TotalA}")

    Y = TotalAy/TotalA
    print(f"Y ={Y}")

    d_conc = y_conc - Y
    d_beam = y_beam - Y
    print(f"d_conc ={d_conc}, d_beam = {d_beam}")

    IAd2_conc = Ict + (Act * d_conc**2)
    IAd2_beam = Ibeam + (Ab * d_beam**2)
    print(f"IAd2_conc = {IAd2_conc}, IAd2_beam = {IAd2_beam}")

    Itr = IAd2_beam + IAd2_conc

    return Itr

def Calc_Cf(Vbeam : float, Vcon : float):
    return min(Vbeam,Vcon)

def Calc_I_eff(Ibeam : float, TotalQn : float, Cf : float, Itr : float)-> float:
    ratiocomposite = (TotalQn / Cf)

    if ratiocomposite < 0.25:
        ratiocomposite = 0.25

    Ieff = Ibeam + ratiocomposite**0.5 * (Itr - Ibeam)

    return Ieff


